In [1]:
import pyodbc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import gc
import datetime
from ipywidgets import IntProgress
from ete3 import Tree, TreeStyle
import swifter

In [2]:
gc.collect()

19

In [3]:
server = 'analys-s'
driver = '{SQL Server}'

connstr = "DRIVER = {0};SERVER = {1};".replace(' ', '').format(driver, server)
dbConnect = pyodbc.connect(connstr)
cursor = dbConnect.cursor()

In [4]:
df = pd.DataFrame({'ID':[], 'City':[], 'Number':[]})

with pyodbc.connect(connstr) as cnxn:    
    quer_str = 'SELECT DISTINCT CONCAT(CURDISCOUNTCARDTYPE, CURDISCOUNTCARDID) as ID,  City, NUMBER\
                   FROM ##CLearClientChks AS c'
    
    df = pd.read_sql(quer_str, cnxn)

In [5]:

df.City = df.swifter.apply(lambda x: 'Пункт 1' if(x.City == 'ИНТЕРНЕТ') else ( 'Пункт 2' if(x.City == 'МОСКВА') else ( 'Пункт 3' if(x.City == 'Санкт-Петербург') else ('Пнукт 4'))), axis = 1)

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
df.City.unique()

array(['Пункт 1', 'Пнукт 4', 'Пункт 2', 'Пункт 3'], dtype=object)

In [14]:
df.to_csv('CustomerRoadMap.csv', index = False)

In [7]:
df.head()

,ID,City,NUMBER
0,01980069,Пункт 1,1
1,02158875,Пункт 1,3
2,03041188,Пункт 1,2
3,0400000356,Пнукт 4,1
4,0400000356,Пнукт 4,4


In [8]:
df.shape

(765098, 3)

In [9]:
def exists(it):
    return (it is not None)

In [10]:
class Node:
    def __init__(self, number, city, ids = []):
        self.val = len(ids)
        self.part = 0
        self.main_part = 0
        self.city = city
        self.number = number
        self.ids = ids
        self.nodes = []

    def add_node(self,  number, city,  ids = []):
        self.nodes.append(NonBinTree(number, city, ids))

    def __repr__(self):
        str1 = f" {self.number} - я Покупка из {self.city} "
        str2 = f"с относительной доля {round(self.part*100, 2)}% "
        str3 = f"и общей долей {round(self.main_part*100, 2)}% "
        str4 = ''
        nodes_clr = list(filter(exists, self.nodes))
        if(len(nodes_clr) == 0):
                return "(" + str1 + str2 + str3 + str4 + ")"
        return "((" + ','.join(map(str, nodes_clr)) + '),' + str1 + str2 + str3 + str4 +")"

    

    
#Level-order traversal 
def levelOrder(root):
    levels = []
    queue = [root]
    while queue and root:
        currNode,nextLevel = [],[]
        for nd in queue:
            if(nd):
                currNode.append(nd)
                for child in nd.nodes:
                    nextLevel.append(child)
        queue = nextLevel
        levels.append(currNode)
    return levels


def FillNode(ds, city, n, ids, root_val, max_n = 4):
    node = None
    prt = 0
    main_prt = 0
    if(n > max_n):
        return node
    if(city == 'НЕ Пункт 1'):
        if(n == max_n):
            strq = 'City != "Пункт 1"  and (NUMBER >= {1}) and (ID in {2})'.format(city, n, ids.reshape(1, -1)[0].tolist())
            part = ds.query(strq)
            strq = '(NUMBER >= {1}) and (ID in {2})'.format(city, n, ids.reshape(1, -1)[0].tolist())
            all_count = len(ds.query(strq))
            prt = len(part.ID)/ all_count 
            main_prt = len(part.ID) / root_val
        else:
            strq = 'City != "Пункт 1"  and (NUMBER == {1}) and (ID in {2})'.format(city, n, ids.reshape(1, -1)[0].tolist())
            part = ds.query(strq)
            strq = '(NUMBER == {1}) and (ID in {2})'.format(city, n, ids.reshape(1, -1)[0].tolist())
            all_count = len(ds.query(strq).ID.unique())
            prt = len(part.ID.unique())/ all_count 
            main_prt = len(part.ID.unique()) / root_val
            
    elif(n != 0):
            strq = 'City == "{0}"  and (NUMBER == {1}) and (ID in {2})'.format(city, n, ids.reshape(1, -1)[0].tolist())
            part = ds.query(strq)
            strq = '(NUMBER == {1}) and (ID in {2})'.format(city, n, ids.reshape(1, -1)[0].tolist())
            all_count = len(ds.query(strq).ID.unique())
            prt = len(part.ID.unique())/ all_count 
            main_prt = len(part.ID.unique()) / root_val
    elif(n == 0):
        part = ds
        all_count = len(part.ID.unique())
        prt = len(part.ID.unique())/ all_count 
        main_prt = len(part.ID.unique()) / root_val
    if(len(part) == 0):
        return None
    node = Node(n, city, part.ID.unique())
    node.part = prt
    node.main_part = main_prt
    if(n > 0):
        for city in ['Пункт 1', 'НЕ Пункт 1']:
            node.nodes.append(FillNode(ds, city, n + 1, part.ID.unique(), root_val))
    else:
        for city in list(ds.City.unique()):
            node.nodes.append(FillNode(ds, city, n + 1, part.ID.unique(), root_val))
    
        
    return node
    


In [11]:
ds = df
ds

,ID,City,NUMBER
0,01980069,Пункт 1,1
1,02158875,Пункт 1,3
2,03041188,Пункт 1,2
3,0400000356,Пнукт 4,1
4,0400000356,Пнукт 4,4
...,...,...,...
765093,9828913537,Пнукт 4,3
765094,9828934691,Пнукт 4,1
765095,9829206954,Пнукт 4,2
765096,9829206954,Пнукт 4,5


In [12]:
root = FillNode(ds, 'ALL', 0, ds.ID.unique(), len(ds.ID.unique()))

In [13]:

t = Tree( str(root).replace('[', '').replace(']', '')+";" )

ts = TreeStyle()
ts.show_leaf_name = True
ts.branch_vertical_margin = 20 # 10 pixels between adjacent branches
#t.render("mytree.png", tree_style=ts, dpi = 300)
ts.scale =  120 # 120 pixels per branch length unit
t.show(tree_style=ts)

# Test Visualization


In [ ]:
a = Node(1, 'ИНТЕРНЕТ')
a.nodes.append(Node(2, 'ИНТЕРНЕТ'))
a.nodes.append(Node(2, 'МОСКВА'))
a.nodes.append(Node(2, 'СПБ'))
a.nodes.append(None)
a.nodes.append(None)



a.nodes[0].nodes.append(Node(3, 'ИНТЕРНЕТ'))
a.nodes[0].nodes.append(Node(3, 'МОСКВА'))
a.nodes[0].nodes.append(Node(3, 'СПБ'))
a.nodes[0].nodes.append(None)
a.nodes[0].nodes.append(None)


print(a)

In [100]:
str(a)

'((((( 3 - я Покупка из ИНТЕРНЕТ с долей относительно данного сегмента 0% и с долей отностильно всех покупок 0%_0 ),( 3 - я Покупка из МОСКВА с долей относительно данного сегмента 0% и с долей отностильно всех покупок 0%_0 ),( 3 - я Покупка из СПБ с долей относительно данного сегмента 0% и с долей отностильно всех покупок 0%_0 )), 2 - я Покупка из ИНТЕРНЕТ с долей относительно данного сегмента 0% и с долей отностильно всех покупок 0%_0 ),( 2 - я Покупка из МОСКВА с долей относительно данного сегмента 0% и с долей отностильно всех покупок 0%_0 ),( 2 - я Покупка из СПБ с долей относительно данного сегмента 0% и с долей отностильно всех покупок 0%_0 )), 1 - я Покупка из ИНТЕРНЕТ с долей относительно данного сегмента 0% и с долей отностильно всех покупок 0%_0 )'

In [101]:

t = Tree( str(a)+";" )

ts = TreeStyle()
ts.show_leaf_name = True
ts.branch_vertical_margin = 20 # 10 pixels between adjacent branches
#t.render("mytree.png", tree_style=ts, dpi = 300)
ts.scale =  120 # 120 pixels per branch length unit
t.show(tree_style=ts)

In [119]:
ds.ID.unique()

array(['01009580', '01051044', '02158875', ..., '9800545785',
       '9800586924', '9801056788'], dtype=object)